<a href="https://colab.research.google.com/github/arthurcornelio88/stable-audio-tools-sam/blob/main/scrapping/scrapping-analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np

# Read the Excel file
url = 'https://github.com/arthurcornelio88/stable-audio-tools-sam/raw/main/scrapping/Export%2010k.xlsx'
df = pd.read_excel(url)

# Write the CSV file (optional parameters)
df.to_csv('your_file.csv', index=False)  # index=False to exclude row numbers


In [7]:
df.head()

,Title,Link to unique page,Image,artist_url,label,Length,Keywords
0,Dancing under the stars. Background music for ...,https://pixabay.com/music/beats-dancing-under-...,https://cdn.pixabay.com/audio/2024/07/17/16-48...,https://pixabay.com/users/white_records-32584949/,White_Records,0:25,background
1,In Slow Motion (Inspiring Ambient Lounge),https://pixabay.com/music/future-bass-in-slow-...,https://cdn.pixabay.com/audio/2024/06/25/10-06...,https://pixabay.com/users/soundbay-601514/,soundbay,1:58,future bass\nabstract
2,Dancing under the stars. Background music for ...,https://pixabay.com/music/beats-dancing-under-...,https://cdn.pixabay.com/audio/2024/07/17/16-48...,https://pixabay.com/users/white_records-32584949/,White_Records,1:06,background
3,Dancing under the stars. Background music for ...,https://pixabay.com/music/beats-dancing-under-...,https://cdn.pixabay.com/audio/2024/07/17/16-48...,https://pixabay.com/users/white_records-32584949/,White_Records,0:48,background
4,Dancing under the stars. background music for ...,https://pixabay.com/music/beats-dancing-under-...,https://cdn.pixabay.com/audio/2024/07/17/16-26...,https://pixabay.com/users/white_records-32584949/,White_Records,2:12,background


In [24]:
'''
# Convert Length column to numeric format after removing non-numeric characters and colons
df['Length'] = df['Length'].astype(str).str.replace(r'[^\d:]', '', regex=True)
df['Length'] = pd.to_numeric(df['Length'].astype(str).str.replace(':', '', regex=False), errors='coerce')

# Impute missing values in `Length` with the mean
mean_length = df['Length'].mean()
df['Length'] = df['Length'].fillna(mean_length)
'''

# Filter the DataFrame to keep only rows where the `Length` value does not match the pattern "\d+:\d+" or "\d+"
problematic_rows = df[~df['Length'].astype(str).str.replace(r'[^\d:]', '', regex=True).str.match(r'^\d+:\d+$|^\d+$')]

# If there are any rows matching the filter criteria, print these rows and their count
if not problematic_rows.empty:
    print(f"\n{len(problematic_rows)} problematic rows with invalid Length values:")
    print(problematic_rows.to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("\nNo problematic rows found.")



1 problematic rows with invalid Length values:
| Title                                            | Link to unique page                                                                           | Image                                                             | artist_url                                 | label   | Length   | Keywords   |
|:-------------------------------------------------|:----------------------------------------------------------------------------------------------|:------------------------------------------------------------------|:-------------------------------------------|:--------|:---------|:-----------|
| Electronic Science-Fiction and Fantasy Orchestra | https://pixabay.com/music/main-title-electronic-science-fiction-and-fantasy-orchestra-187063/ | https://cdn.pixabay.com/audio/2024/01/19/16-28-00-248_200x200.jpg | https://pixabay.com/users/juliush-3921568/ | JuliusH | nan      | fantasy    |
|                                                  |            

In [25]:
problematic_rows.head()

,Title,Link to unique page,Image,artist_url,label,Length,Keywords
5904,Electronic Science-Fiction and Fantasy Orchestra,https://pixabay.com/music/main-title-electroni...,https://cdn.pixabay.com/audio/2024/01/19/16-28...,https://pixabay.com/users/juliush-3921568/,JuliusH,NaN,fantasy\norchestra\nmusic


In [19]:
# Print the column names and their data types
print("Column names and datatypes:")
print(df.info())

Column names and datatypes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9930 entries, 0 to 9929
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                9930 non-null   object
 1   Link to unique page  9930 non-null   object
 2   Image                8234 non-null   object
 3   artist_url           9808 non-null   object
 4   label                9808 non-null   object
 5   Length               9930 non-null   int64 
 6   Keywords             9930 non-null   object
dtypes: int64(1), object(6)
memory usage: 543.2+ KB
None


In [13]:
# Calculate and print the number of null values in each column
print("\nNull values per column:")
print(df.isnull().sum().to_markdown(numalign="left", stralign="left"))


Null values per column:
|                     | 0    |
|:--------------------|:-----|
| Title               | 0    |
| Link to unique page | 0    |
| Image               | 1696 |
| artist_url          | 122  |
| label               | 122  |
| Length              | 0    |
| Keywords            | 0    |


In [14]:
# Calculate and print the number of duplicated rows
print(f"\nNumber of duplicated rows: {df.duplicated().sum()}")


Number of duplicated rows: 0


In [15]:
# Calculate and print the number of unique values in each column
print("\nUnique values per column:")
print(df.nunique().to_markdown(numalign="left", stralign="left"))


Unique values per column:
|                     | 0    |
|:--------------------|:-----|
| Title               | 9647 |
| Link to unique page | 9929 |
| Image               | 8233 |
| artist_url          | 818  |
| label               | 818  |
| Length              | 811  |
| Keywords            | 5565 |


In [16]:
# Calculate and print the mean and standard deviation of the `Length` column
mean_length = df['Length'].mean()
std_length = df['Length'].std()
print(f"\nMean Length: {mean_length:.2f} seconds")
print(f"Standard Deviation of Length: {std_length:.2f} seconds")


Mean Length: 3070724604922.63 seconds
Standard Deviation of Length: 305995819146057.75 seconds


In [18]:
# Identify outliers in the `Length` column (using Z-score method)
z_scores = (df['Length'] - mean_length) / std_length
outliers = df[(z_scores > 3) | (z_scores < -3)]

# Print the number of outliers found
if not outliers.empty:
    print(f"\n{len(outliers)} outliers found in Length (using Z-score method):")
    print(outliers.to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("\nNo outliers found in Length (using Z-score method).")


1 outliers found in Length (using Z-score method):
| Title                                            | Link to unique page                                                                           | Image                                                             | artist_url                                 | label   | Length            | Keywords   |
|:-------------------------------------------------|:----------------------------------------------------------------------------------------------|:------------------------------------------------------------------|:-------------------------------------------|:--------|:------------------|:-----------|
| Electronic Science-Fiction and Fantasy Orchestra | https://pixabay.com/music/main-title-electronic-science-fiction-and-fantasy-orchestra-187063/ | https://cdn.pixabay.com/audio/2024/01/19/16-28-00-248_200x200.jpg | https://pixabay.com/users/juliush-3921568/ | JuliusH | 30492295296605903 | fantasy    |
|                                